In [13]:
import torch
from resnet18 import ResNet, data_loader, ResidualBlock

In [14]:
test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

device = torch.device(device)

Files already downloaded and verified


In [15]:
model = ResNet().to(device)
model.load_state_dict(torch.load("model/Model for Inference.pth", map_location=device, weights_only=True))
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

Accuracy of the network on the 10000 test images: 83.43 %
